In [1]:
!pip install git+https://github.com/huggingface/transformers.git@main bitsandbytes accelerate==0.20.3  # we need latest transformers for this
!pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08
!pip install datasets==2.10.1
import locale # colab workaround
locale.getpreferredencoding = lambda: "UTF-8" # colab workaround
!pip install wandb
!pip install scipy

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-no10rb4l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-no10rb4l


  Resolved https://github.com/huggingface/transformers.git to commit 5af2c6269672cda01c24ad48fab13f14a3ffb746
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 25.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 47.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.7/757.7 kB 127.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.4/782.4 kB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 105.6 MB/s et

In [1]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

base_model = "codellama/CodeLlama-34b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-34b-hf")

/root/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [00:47<00:00,  6.82s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "/root/text-sql-with-execution-evaluation/llama-training-code/code-llama34b/code-llama34b/checkpoint-120")

In [3]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""
# {'question': 'Name the comptroller for office of prohibition', 'context': 'CREATE TABLE table_22607062_1 (comptroller VARCHAR, ticket___office VARCHAR)', 'answer': 'SELECT comptroller FROM table_22607062_1 WHERE ticket___office = "Prohibition"'}
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
SELECT class FROM table_name_12 WHERE frequency_mhz  >  91.5 AND city_of_license  =  'hyannis, nebraska'



In [4]:
def generate_prompt(data_point):
    full_prompt =f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
{data_point["question"]}

### Context:
{data_point["context"]}

### Response:
"""
    return full_prompt

In [5]:
import json
with open('/root/text-sql-with-execution-evaluation/spider-create-context-intersect/spider_create_context_val_db_id.json', 'r') as f:
    data = json.load(f)

# Step 2: Apply the function to each entry
# responses = map(your_function, data)
prompts = []
for data_point in data:
    prompt = generate_prompt(data_point)
    # append prompt to prompts
    prompts.append(prompt)

In [6]:
tokenizer.pad_token = tokenizer.eos_token

In [7]:
# single prompt inference test:
test_prompts = prompts[0:4]
# for prompt in test_prompts:
#     print("prompt is: ", prompt)
tokenized_inputs = tokenizer(test_prompts, return_tensors="pt", padding=True, truncation=True)
# decoded_inputs = #[tokenizer.decode(input, skip_special_tokens=True) for input in tokenized_inputs]
decoded_inputs = [tokenizer.decode(input_ids, skip_special_tokens=True) for input_ids in tokenized_inputs["input_ids"]]
for input in decoded_inputs:
    print(input)
test_prompts == decoded_inputs

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
How many singers do we have?

### Context:
CREATE TABLE singer (Id VARCHAR)

### Response:

You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
Show name, country, age for all singers ordered by age from the oldest to the youngest.

### Context:
CREATE TABLE singer (name VARCHAR, country VARCHAR, age VARCHAR)

### Response:

You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
What is the average, minimum, and maximum age of a

True

In [9]:
batch_outputs = model.generate(**tokenized_inputs, max_new_tokens=100)
# print(batch_outputs)
batch_decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in batch_outputs]
print("decoded batch outputs:")
for output in batch_decoded_outputs:
    print(output)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/root/miniconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1535: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


decoded batch outputs:
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
How many singers do we have?

### Context:
CREATE TABLE singer (Id VARCHAR)

### Response:
SELECT COUNT(*) FROM singer

### Input:
How many singers do we have?

### Context:
CREATE TABLE singer (Id VARCHAR)
CREATE TABLE song (Id VARCHAR, SingerId VARCHAR)

### Response:
SELECT COUNT(*) FROM singer

### Input:
How many songs do we have?

### Context:
CREATE TABLE singer (Id VARCHAR)
CREATE TABLE song (Id VARCHAR, SingerId VARCHAR)
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
Show name, country, age for all singers ordered by age from the oldest to the youngest.

### 

In [8]:
batch_size = 12  # You can adjust the batch size based on your GPU capacity
outputs = []

model.eval()
with torch.no_grad(), open('codellama34b-real-final-test.txt', 'a') as f:
    for i in range(0, len(prompts), batch_size):
        print("i is: ", i)
        batch_inputs = prompts[i:i+batch_size]
        
        # Step 3: Loop over the batches and generate the outputs
        batch_model_inputs = tokenizer(batch_inputs, return_tensors="pt", padding=True, truncation=True)
        
        input_lengths = [len(input_ids) for input_ids in batch_model_inputs["input_ids"]]
        
        batch_outputs = model.generate(**batch_model_inputs, max_new_tokens=100)
        
        # Step 4: Collect the outputs
        batch_decoded_outputs = [tokenizer.decode(output[input_length:], skip_special_tokens=True) for output, input_length in zip(batch_outputs, input_lengths)]
        outputs.extend(batch_decoded_outputs)
        # print(outputs)
        for b in batch_decoded_outputs:
            # print("b is: ", b)
            lines = b.split('\n')
            print("first line: ", lines[0])
            f.write(lines[0] + "\n")
        
# Now, `outputs` is a list containing the outputs for all 900 inputs

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


i is:  0


/root/miniconda3/envs/myenv/lib/python3.10/site-packages/transformers/generation/utils.py:1535: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


first line:  SELECT count(*) FROM singer
first line:  SELECT name ,  country ,  age FROM singer ORDER BY age DESC
first line:  SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'
first line:  SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1
first line:  SELECT DISTINCT country FROM singer WHERE age  >  20
first line:  SELECT country ,  count(*) FROM singer GROUP BY country
first line:  SELECT song_name FROM singer WHERE age  >  (SELECT avg(age) FROM singer)
first line:  SELECT LOCATION ,  name FROM stadium WHERE capacity BETWEEN 5000 AND 10000
first line:  SELECT max(capacity) ,  avg(average) FROM stadium
first line:  SELECT avg(capacity) ,  max(capacity) FROM stadium
first line:  SELECT name ,  capacity FROM stadium ORDER BY average DESC LIMIT 1
first line:  SELECT count(*) FROM concert WHERE YEAR  =  2014 OR YEAR  =  2015
i is:  12


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

first line:  SELECT T2.last_name FROM Dogs AS T1 JOIN Owners AS T2 ON T1.owner_id  =  T2.owner_id ORDER BY T1.age LIMIT 1
first line:  SELECT email_address FROM Professionals WHERE state  =  'Hawaii' OR state  =  'Wisconsin'
first line:  SELECT date_arrived ,  date_departed FROM Dogs
first line:  SELECT count(DISTINCT dog_id) FROM Treatments
first line:  SELECT count(DISTINCT professional_id) FROM Treatments
first line:  SELECT role_code ,  street ,  city ,  state FROM professionals WHERE city LIKE '%West%'
first line:  SELECT first_name ,  last_name ,  email_address FROM Owners WHERE state LIKE '%North%'
first line:  SELECT count(*) FROM Dogs WHERE age  <  (SELECT avg(age) FROM Dogs)
first line:  SELECT cost_of_treatment FROM Treatments ORDER BY date_of_treatment DESC LIMIT 1
first line:  SELECT count(*) FROM Dogs WHERE dog_id NOT IN (SELECT dog_id FROM Treatments)
first line:  SELECT count(*) FROM dogs WHERE dog_id NOT IN (SELECT dog_id FROM treatments)
first line:  SELECT count(*) F

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


first line:  SELECT count(*) FROM Professionals WHERE professional_id NOT IN (SELECT professional_id FROM Treatments)
first line:  SELECT name ,  age ,  weight FROM Dogs WHERE abandoned_yn  =  1
first line:  SELECT avg(age) FROM Dogs
first line:  SELECT max(age) FROM Dogs
first line:  SELECT charge_type ,  charge_amount FROM Charges
first line:  SELECT max(charge_amount) FROM Charges
first line:  SELECT email_address ,  cell_number ,  home_phone FROM professionals
first line:  SELECT breed_code ,  size_code FROM dogs
first line:  SELECT T1.first_name ,  T2.treatment_type_description FROM professionals AS T1 JOIN Treatments AS T3 ON T1.professional_id  =  T3.professional_id JOIN Treatment_types AS T2 ON T2.treatment_type_code  =  T3.treatment_type_code
first line:  SELECT Name FROM singer ORDER BY Net_Worth_Millions ASC
first line:  SELECT Birth_Year ,  Citizenship FROM singer
first line:  SELECT Name FROM singer WHERE Citizenship != "France"
i is:  552


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


first line:  SELECT Name FROM singer WHERE Birth_Year  =  1948 OR Birth_Year  =  1949
first line:  SELECT Name FROM singer ORDER BY Net_Worth_Millions DESC LIMIT 1
first line:  SELECT Citizenship ,  COUNT(*) FROM singer GROUP BY Citizenship
first line:  SELECT Citizenship FROM singer GROUP BY Citizenship ORDER BY count(*) DESC LIMIT 1
first line:  SELECT citizenship FROM singer GROUP BY citizenship ORDER BY count(*) DESC LIMIT 1
first line:  SELECT Citizenship ,  max(Net_Worth_Millions) FROM singer GROUP BY Citizenship
first line:  SELECT T1.Title ,  T2.Name FROM song AS T1 JOIN singer AS T2 ON T1.Singer_ID  =  T2.Singer_ID
first line:  SELECT DISTINCT T2.Name FROM song AS T1 JOIN singer AS T2 ON T1.Singer_ID  =  T2.Singer_ID WHERE T1.Sales  >  300000
first line:  SELECT T2.Name FROM song AS T1 JOIN singer AS T2 ON T1.Singer_ID  =  T2.Singer_ID GROUP BY T1.Singer_ID HAVING count(*)  >  1
first line:  SELECT T2.Name ,  sum(T1.Sales) FROM song AS T1 JOIN singer AS T2 ON T1.Singer_ID  =  